## I. Scrape page

Import Libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd

/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Scrape Table Cells

In [2]:
url='https://www.kprichitoregalos.com.co/codigo-postal/'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

Check the length of the first 10 rows

In [3]:
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Parse Table Header

In [4]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' % (i,name))
    col.append((name,[]))

1:"CODIGO POSTAL"
2:"LOCALIDAD"
3:"BARRIOS CONTENIDOS EN EL CODIGO POSTAL"


Name of columns

In [5]:
col

[('CODIGO POSTAL', []),
 ('LOCALIDAD', []),
 ('BARRIOS CONTENIDOS EN EL CODIGO POSTAL', [])]

Creating Pandas DataFrame

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Check if all columns have the same number of rows

In [7]:
[len(C) for (title,C) in col]

[63, 63, 63]

Create the DataFrame

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head(10)

,BARRIOS CONTENIDOS EN EL CODIGO POSTAL,CODIGO POSTAL,LOCALIDAD
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa
5,"Antonia Santos, Argelia II, Bosa, El Jardín, G...",110731,Bosa
6,"Andalucía II, Cementerio Jardínes Apogeo, Char...",110741,Bosa
7,"Belén, Centro Administrativo, Egipto, La Cated...",111711,Candelaria
8,"Esperanza, Páramo, Páramo I, Páramo II, Páramo...",110211,Chapinero
9,"Antiguo Country, Bellavista, Chicó Norte, Chic...",110221,Chapinero


Number of rows and columns of dataframe

In [9]:
df.shape

(63, 3)

## III. Latitude and the longitude coordinates of each neighborhood

In [10]:
# Install packages
!pip install pgeocode

You should consider upgrading via the 'pip install --upgrade pip' command.


### i. Explore Dataset

Filter the codes of Colombia and then Bogota

In [11]:
import pgeocode
bog = pgeocode.Nominatim('CO')

In [12]:
# Try with the postal codes of Bogota
bog=bog.query_postal_code(['111511', '111211', '111221', '110711', '110721', '110731', '110741', '111711', '110211', '110221', '110231', '111911', '111921', '111931', '111941', '111951', '111961', '111971', '111011', '111021', '111031', '111041', '111051', '111061', '111071', '110911', '110921', '110931', '110811', '110821', '110831', '110841', '110851', '110861', '110871', '110881', '111411', '111611', '111621', '111631', '111811', '111821', '111831', '111841', '110411', '110421', '110431', '110441', '110311', '110321', '111111', '111121', '111131', '111141', '111151', '111156', '111161', '111166', '111171', '111176', '112011', '112021', '112031', '112041', '111311', '111321', '110611', '110621', '110111', '110121', '110131', '110141', '110151', '110511', '110521', '110531', '110541', '110551','110571'])
bog.head()

,postal_code,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,111511,CO,Antonio Nariño,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.5889,-74.1030,4
1,111211,CO,Barrios Unidos,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6770,-74.0692,4
2,111221,CO,Barrios Unidos,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6617,-74.0780,4
3,110711,CO,Bosa,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6369,-74.1885,4
4,110721,CO,Bosa,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6280,-74.2055,4


Change the names of variables

In [13]:
df_codes=df.rename(columns = {'BARRIOS CONTENIDOS EN EL CODIGO POSTAL':'Neighbourhood', 'CODIGO POSTAL':'postal_code', 'LOCALIDAD':'Borough'})
df_codes.head()

,Neighbourhood,postal_code,Borough
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa


Merge two data bases

In [14]:
final_base = pd.merge(df_codes, bog, how = "left", on = "postal_code")
final_base.head()

,Neighbourhood,postal_code,Borough,country code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño,CO,Antonio Nariño,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.5889,-74.1030,4
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos,CO,Barrios Unidos,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6770,-74.0692,4
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos,CO,Barrios Unidos,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6617,-74.0780,4
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa,CO,Bosa,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6369,-74.1885,4
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa,CO,Bosa,"Bogota, D.C.",34,Bogotá D.C.,11001,NaN,NaN,4.6280,-74.2055,4


In [15]:
final_base.shape

(63, 14)

Keep only variables of interes

In [16]:
final_base=final_base.drop(['country code', 'place_name', 'state_name', 'state_code', 'county_name', 'county_code', 'community_name', 'community_code', 'accuracy'], axis=1)

In [17]:
final_base=final_base.rename(columns = {'latitude':'Latitude', 'longitude':'Longitude', 'postal_code':'Postcode'})
final_base.head()

,Neighbourhood,Postcode,Borough,Latitude,Longitude
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño,4.5889,-74.1030
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos,4.6770,-74.0692
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos,4.6617,-74.0780
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa,4.6369,-74.1885
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa,4.6280,-74.2055


.............................................................

## IV. Explore and cluster the neighborhoods in Bogota

First, let's download all the dependencies that we will need

In [18]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_420/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Libraries imported.


### i. Explore Dataset

In [19]:
base_bogota = final_base
base_bogota.head()

,Neighbourhood,Postcode,Borough,Latitude,Longitude
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño,4.5889,-74.1030
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos,4.6770,-74.0692
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos,4.6617,-74.0780
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa,4.6369,-74.1885
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa,4.6280,-74.2055


### ii. Create a map of Bogota with neighborhoods superimposed on top

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>m_explorer</em>, as shown below.

In [20]:
address = 'Bogota, BOG'

geolocator = Nominatim(user_agent="m_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bogota are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bogota are 4.700912450000001, -74.14606074859216.


In [21]:
# create map of Bogotá using latitude and longitude values
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(base_bogota['Latitude'], base_bogota['Longitude'], base_bogota['Borough'], base_bogota['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)  
    
map_bogota

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

### iii. Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'QY2KJJE52HKB4B5W2ZQRC1PYQZQAJIHRDLATVHSDURMAY0XU' # your Foursquare ID
CLIENT_SECRET = 'DCM2JYNYQUUCJMQQJT2KTYIZHGCSOY5GMRAPULQTNM3BZL33' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QY2KJJE52HKB4B5W2ZQRC1PYQZQAJIHRDLATVHSDURMAY0XU
CLIENT_SECRET:DCM2JYNYQUUCJMQQJT2KTYIZHGCSOY5GMRAPULQTNM3BZL33


### iv. Let's explore the first neighborhood in our dataframe

Get the neighborhood's name

In [23]:
base_bogota.loc[0, 'Neighbourhood']

'Caracas, Ciudad Berna, Ciudad Jardín Sur, Eduardo Frey, La Fragua, La Fraguita, La Hortúa, Policarpa, Restrepo, Restrepo Occidental, San Antonio, Santander, Santander Sur, Sena, Sevilla, Villa Mayor Oriental'

Get the neighborhood's latitude and longitude values

In [24]:
neighborhood_latitude = base_bogota.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = base_bogota.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = base_bogota.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Caracas, Ciudad Berna, Ciudad Jardín Sur, Eduardo Frey, La Fragua, La Fraguita, La Hortúa, Policarpa, Restrepo, Restrepo Occidental, San Antonio, Santander, Santander Sur, Sena, Sevilla, Villa Mayor Oriental are 4.5889, -74.10300000000002.


### v. Now, let's get the top 100 venues that are in Borough within a radius of 500 meters

First, let's create the GET request URL. Name your URL url.

In [25]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QY2KJJE52HKB4B5W2ZQRC1PYQZQAJIHRDLATVHSDURMAY0XU&client_secret=DCM2JYNYQUUCJMQQJT2KTYIZHGCSOY5GMRAPULQTNM3BZL33&v=20180605&ll=4.5889,-74.10300000000002&radius=500&limit=100'

Send the GET request and examine the resutls

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4d6a639388d7001ba07a8c'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4eac5ce830f823f86eb3a5df-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pieshop_',
          'suffix': '.png'},
         'id': '52e81612bcbc57f1066b7a0a',
         'name': 'Pie Shop',
         'pluralName': 'Pie Shops',
         'primary': True,
         'shortName': 'Pie Shop'}],
       'id': '4eac5ce830f823f86eb3a5df',
       'location': {'address': 'Cr 19 15-19 S',
        'cc': 'CO',
        'city': 'Bogotá',
        'country': 'Colombia',
        'distance': 380,
        'formattedAddress': ['Cr 19 15-19 S',
         'Bogotá, Bogotá D.C.',
         'Colombia'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 4.586891048300204,
     

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pasteleria Guernika,Pie Shop,4.586891,-74.100228
1,sport body,Gym,4.587048,-74.099812
2,Kokoriko,Fried Chicken Joint,4.586530,-74.100282
3,Wenbrulee,Restaurant,4.585942,-74.099846
4,Farmatodo,Convenience Store,4.585889,-74.102457


And how many venues were returned by Foursquare?

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


## V. Explore Neighborhoods in Bogota

Let's create a function to repeat the same process to all the neighborhoods in Bogota

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called bogota_venues

In [31]:
bogota_venues = getNearbyVenues(names=base_bogota['Neighbourhood'],
                                   latitudes=base_bogota['Latitude'],
                                   longitudes=base_bogota['Longitude']
                                  )

Caracas, Ciudad Berna, Ciudad Jardín Sur, Eduardo Frey, La Fragua, La Fraguita, La Hortúa, Policarpa, Restrepo, Restrepo Occidental, San Antonio, Santander, Santander Sur, Sena, Sevilla, Villa Mayor Oriental
Alcázares Norte, Doce de Octubre, Entrerríos, Escuela Militar, Jorge Eliecer Gaitán, Juan XXIII, La Aurora, La Castellana, La Libertad, La Patria, Los Andes, Metrópolis, Once de Noviembre, Polo Club, Rionegro, San Felipe, San Fernando Occidental, Santa So
Alcázares, Baquero, Benjamín Herrera, Colombia, Concepción Norte, El Rosario, José Joaquín Vargas, La Esperanza, La Merced Norte, La Paz, Muequeta, Parque Distrital Salitre, Parque Popular Salitre, Popular Modelo, Quinta Mutis, Rafael Uribe, San Fernando
Betania, Brasil, Brasilia, Cañaveralejo, Cañaveralejo Rural, Chicalá, El Corzo, El Corzo Rural, El Danubio Azul, El Portal del Brasil, Escocia, Escocia, La Cabaña, La Libertad, Osorio X, Osorio X Urbano, Osorio XXIII, Parcela El Porvenir, San Martín
Bosa Nova, Bosa Nova El Porveni

Aguas Claras, Buenos Aires, Calvo Sur, El Triángulo, Hoya San Cristóbal, La María, Las Brisas, Los Laureles I, Modelo Sur, Molino Rural, Molinos de Oriente, Monte Carlo, Nariño Sur, Primero de Mayo, Quinta Ramos, San Cristóbal Sur, San Javier, Santa Ana S
Atenas Sur, Bello Horizonte, Córdoba, Granada Sur, Las Mercedes, Los Alpes, Montebello, Ranajal, San Blas, San Blas II, San Isidro, San Pedro, San Vicente, Santa Inés Sur, Suramérica, Veinte de Julio, Villa de Los Alapes, Villa de Los Alapes I
Altamira, Altos del Poblado, Altos del Zipa, Altos del Zuque, Barcelona Sur, Bellavista Sur, Bosque de Los Alpes, La Arboleda, La Gloria Occidental, La Gloria Oriental, La Victoria, Las Guacamayas, Las Lomas, Moralba, Puente Colorado, Quindío
Canadá o Guira, Chiguaza, Ciudad Londres I, El Paraíso, El Pinar, Juan Rey (La Paz), La Belleza, Las Gaviotas, Los Libertadores, Nueva Delhí, Nueva Gloria, San Rafael Usme, Santa Rita Sur Oriental, Villabel, Yomasa
Bosque Izquierdo, Hoya Teusaca, La Alameda

Let's check the size of the resulting dataframe

In [32]:
print(bogota_venues.shape)
bogota_venues.head()

(342, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",4.5889,-74.103,Pasteleria Guernika,4.586891,-74.100228,Pie Shop
1,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",4.5889,-74.103,sport body,4.587048,-74.099812,Gym
2,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",4.5889,-74.103,Kokoriko,4.586530,-74.100282,Fried Chicken Joint
3,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",4.5889,-74.103,Wenbrulee,4.585942,-74.099846,Restaurant
4,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",4.5889,-74.103,Farmatodo,4.585889,-74.102457,Convenience Store


Let's check how many venues were returned for each neighborhood

In [33]:
bogota_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alamos, El Cedro, El Madrigal, El Muelle, Garcés Navas, Garcés Navas Oriental, Los Ángeles, San Antonio Engativá, Santa Mónica, Villa Amalia, Villa del Mar, Villa Sagrario",2,2,2,2,2,2
"Alcalá, Alqueria, Autopista Sur, Galán, La Camelia, Ospina Pérez, Ospina Pérez Sur, Provivienda Norte, San Eusebio, Tejar",4,4,4,4,4,4
"Alcázares Norte, Doce de Octubre, Entrerríos, Escuela Militar, Jorge Eliecer Gaitán, Juan XXIII, La Aurora, La Castellana, La Libertad, La Patria, Los Andes, Metrópolis, Once de Noviembre, Polo Club, Rionegro, San Felipe, San Fernando Occidental, Santa So",4,4,4,4,4,4
"Alcázares, Baquero, Benjamín Herrera, Colombia, Concepción Norte, El Rosario, José Joaquín Vargas, La Esperanza, La Merced Norte, La Paz, Muequeta, Parque Distrital Salitre, Parque Popular Salitre, Popular Modelo, Quinta Mutis, Rafael Uribe, San Fernando",9,9,9,9,9,9
"Almirante Colon, Altos de Chozica, Andes Norte, Ciudad Hunza, Club de Los Lagartos, Julio Flórez, Los Naranjos, Niza Norte, Niza Suba, Niza Sur, Potosí, Santa Rosa, Vereda Suba Naranjos",6,6,6,6,6,6
"Alqueria La Fragua, Alqueria La Fragua Norte, Boita, La Campiña, Las Delicias, Nueva York, Provivienda, Provivienda Occidental, Renania Urapanes, Santa Catalina, Timiza, Timiza A, Timiza B, Timiza C, Tundama",6,6,6,6,6,6
"Altos de Suba, Atenas, Britalia, Cantagallo, Casa Blanca Suba I, Casablanca Suba, Delmonte, El Plan, Escuela de Carabineros, Gilmar, Granada Norte, Iragua, Mazuren, Portales del Norte, Prado Pinzón, San José V Sector, Santa Helena, Suba Cerros, Suba Urban",11,11,11,11,11,11
"Andalucía II, Cementerio Jardínes Apogeo, Charles de Gaulle, El Retazo, Gualoche, José Antonio Galán, José María Carbonel, La Estación Bosa, Nueva Granada Bosa, Olarte, San Pablo Bosa, Villa Anny I, Villa Anny II, Villa del Río",3,3,3,3,3,3
"Antiguo Country, Bellavista, Chicó Norte, Chicó Norte II Sector, Chicó Norte III Sector, El Bagazal, El Chicó, El Nogal, El Refugio, El Retiro, Espartillal, La Cabrera, Lago Gaitán, Los Rosales, Porciuncula, Seminario",4,4,4,4,4,4


Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(bogota_venues['Venue Category'].unique())))

There are 109 uniques categories.


## VI. Analyze Each Neighborhood

In [35]:
# one hot encoding
bogota_onehot = pd.get_dummies(bogota_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bogota_onehot['Neighborhood'] = bogota_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bogota_onehot.columns[-1]] + list(bogota_onehot.columns[:-1])
bogota_onehot = bogota_onehot[fixed_columns]

bogota_onehot.head()

,Wings Joint,Airport,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Bowling Alley,Breakfast Spot,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Cultural Center,Department Store,Donut Shop,Drugstore,Falafel Restaurant,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Nightclub,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Rental Car Location,Restaurant,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,South American Restaurant,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Theater,Theme Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size

In [36]:
bogota_onehot.shape

(342, 109)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
bogota_grouped = bogota_onehot.groupby('Neighborhood').mean().reset_index()
bogota_grouped

,Neighborhood,Wings Joint,Airport,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bookstore,Bowling Alley,Breakfast Spot,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Creperie,Cultural Center,Department Store,Donut Shop,Drugstore,Falafel Restaurant,Fast Food Restaurant,Food,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Nightclub,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pub,Rental Car Location,Restaurant,Rock Club,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Snack Place,Soccer Field,South American Restaurant,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Theater,Theme Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant
0,"Alamos, El Cedro, El Madrigal, El Muelle, Garc...",0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
1,"Alcalá, Alqueria, Autopista Sur, Galán, La Cam...",0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.250000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.25,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.250000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00
2,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.25,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.500000,0.000000,0.000,0.000000,0.000000,0.0

Let's confirm the new size

In [38]:
bogota_grouped.shape

(47, 109)

Let's print each neighborhood along with the top 10 most common venues

In [39]:
num_top_venues = 10

for hood in bogota_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bogota_grouped[bogota_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alamos, El Cedro, El Madrigal, El Muelle, Garcés Navas, Garcés Navas Oriental, Los Ángeles, San Antonio Engativá, Santa Mónica, Villa Amalia, Villa del Mar, Villa Sagrario----
                           venue  freq
0                    Pizza Place   0.5
1                   Burger Joint   0.5
2                    Wings Joint   0.0
3          Performing Arts Venue   0.0
4  Paper / Office Supplies Store   0.0
5           Other Great Outdoors   0.0
6                      Nightclub   0.0
7                Nature Preserve   0.0
8                    Music Venue   0.0
9                    Music Store   0.0


----Alcalá, Alqueria, Autopista Sur, Galán, La Camelia, Ospina Pérez, Ospina Pérez Sur, Provivienda Norte, San Eusebio, Tejar----
                           venue  freq
0     Construction & Landscaping  0.25
1                   Burger Joint  0.25
2                      BBQ Joint  0.25
3           Fast Food Restaurant  0.25
4                    Wings Joint  0.00
5                       M

                  venue  freq
0     Mobile Phone Shop  0.25
1       Nature Preserve  0.25
2         Shopping Mall  0.25
3                  Park  0.25
4           Wings Joint  0.00
5  Other Great Outdoors  0.00
6             Nightclub  0.00
7           Music Venue  0.00
8           Music Store  0.00
9                Museum  0.00


----Aures, Costa Azul, El Poa, El Rincón, El Rincón Norte, La Chucua, Lago de Suba, Potrerillo, Rincón de Suba, San Cayetano, Santa Teresa de Suba, Ttes de Colombia, Villa Elisa----
                           venue  freq
0                           Park   0.5
1                    Auto Garage   0.5
2              Mobile Phone Shop   0.0
3  Paper / Office Supplies Store   0.0
4           Other Great Outdoors   0.0
5                      Nightclub   0.0
6                Nature Preserve   0.0
7                    Music Venue   0.0
8                    Music Store   0.0
9                         Museum   0.0


----Autopista Medellín, Bochica, Ciudad Bachué, Ciudad 

                  venue  freq
0               Airport  0.25
1           Candy Store  0.25
2                 Plaza  0.25
3                  Lake  0.25
4       Motorcycle Shop  0.00
5  Other Great Outdoors  0.00
6             Nightclub  0.00
7       Nature Preserve  0.00
8           Music Venue  0.00
9           Music Store  0.00


----Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, Ciudadela El Recreo I, Ciudadela El Recreo II, El Remanso, El Remanso I, La Independencia, La Paz Bosa, La Vega San Bernardino, Las Margaritas, Los Sauces, Remanso Urbano, San Antonio, San Bernardino II----
                           venue  freq
0     Construction & Landscaping   1.0
1                    Wings Joint   0.0
2                Motorcycle Shop   0.0
3  Paper / Office Supplies Store   0.0
4           Other Great Outdoors   0.0
5                      Nightclub   0.0
6                Nature Preserve   0.0
7                    Music Venue   0.0
8                    Music Store   0.0
9                     

                           venue  freq
0                           Park   0.5
1               Arepa Restaurant   0.5
2              Mobile Phone Shop   0.0
3  Paper / Office Supplies Store   0.0
4           Other Great Outdoors   0.0
5                      Nightclub   0.0
6                Nature Preserve   0.0
7                    Music Venue   0.0
8                    Music Store   0.0
9                         Museum   0.0




#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bogota_grouped['Neighborhood']

for ind in np.arange(bogota_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bogota_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alamos, El Cedro, El Madrigal, El Muelle, Garc...",Pizza Place,Burger Joint,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
1,"Alcalá, Alqueria, Autopista Sur, Galán, La Cam...",Construction & Landscaping,Burger Joint,BBQ Joint,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Drugstore,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall
2,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",Restaurant,Mexican Restaurant,Music Venue,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
3,"Alcázares, Baquero, Benjamín Herrera, Colombia...",Soccer Field,Astrologer,Clothing Store,Bakery,Auto Garage,Creperie,Motorcycle Shop,Department Store,Cultural Center,Vegetarian / Vegan Restaurant
4,"Almirante Colon, Altos de Chozica, Andes Norte...",Park,Scenic Lookout,Other Great Outdoors,Bus Stop,Burger Joint,Gym,Art Museum,Chinese Restaurant,Cocktail Bar,Coffee Shop


## VII. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [42]:
# set number of clusters
kclusters = 5

bogota_grouped_clustering = bogota_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bogota_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 0, 4, 4, 4, 4, 0, 4, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

But first, I am going to change the name of one column

In [43]:
base_bogota=base_bogota.rename(columns = {'Neighbourhood':'Neighborhood'})

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bogota_merged = base_bogota

# merge bogota_grouped with bogota_data to add latitude/longitude for each neighborhood
bogota_merged = bogota_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bogota_merged.head() # check the last columns!

,Neighborhood,Postcode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Caracas, Ciudad Berna, Ciudad Jardín Sur, Edua...",111511,Antonio Nariño,4.5889,-74.1030,4.0,Restaurant,BBQ Joint,Department Store,Seafood Restaurant,Shopping Mall,Pizza Place,Pie Shop,Sandwich Place,Donut Shop,Drugstore
1,"Alcázares Norte, Doce de Octubre, Entrerríos, ...",111211,Barrios Unidos,4.6770,-74.0692,0.0,Restaurant,Mexican Restaurant,Music Venue,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
2,"Alcázares, Baquero, Benjamín Herrera, Colombia...",111221,Barrios Unidos,4.6617,-74.0780,4.0,Soccer Field,Astrologer,Clothing Store,Bakery,Auto Garage,Creperie,Motorcycle Shop,Department Store,Cultural Center,Vegetarian / Vegan Restaurant
3,"Betania, Brasil, Brasilia, Cañaveralejo, Cañav...",110711,Bosa,4.6369,-74.1885,4.0,Shopping Mall,Movie Theater,Vegetarian / Vegan Restaurant,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
4,"Bosa Nova, Bosa Nova El Porvenir, Chicó Sur, C...",110721,Bosa,4.6280,-74.2055,1.0,Construction & Landscaping,Vegetarian / Vegan Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie


## VIII. Examine Clusters

We examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [45]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 0, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,111211,0.0,Restaurant,Mexican Restaurant,Music Venue,Caribbean Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
6,110741,0.0,Mexican Restaurant,Fried Chicken Joint,Restaurant,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
12,111921,0.0,Park,Italian Restaurant,Burger Joint,Restaurant,Department Store,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall
22,111051,0.0,Historic Site,Bakery,Park,Restaurant,Sandwich Place,Gym,Gastropub,Grocery Store,Clothing Store,Cocktail Bar
37,111611,0.0,Park,Hotel,Italian Restaurant,BBQ Joint,Bar,Drugstore,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall
39,111631,0.0,Park,Mexican Restaurant,Pizza Place,Restaurant,Bar,Vegetarian / Vegan Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant
41,111821,0.0,Park,Arepa Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
52,111131,0.0,Park,Auto Garage,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
53,111141,0.0,Park,Nature Preserve,Shopping Mall,Mobile Phone Shop,Vegetarian / Vegan Restaurant,Department Store,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant
54,111151,0.0,Park,Airport,Business Service,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping


#### Cluster 2

In [46]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 1, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,110721,1.0,Construction & Landscaping,Vegetarian / Vegan Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Creperie
35,110881,1.0,Construction & Landscaping,Bakery,Bar,Vegetarian / Vegan Restaurant,Falafel Restaurant,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store


#### Cluster 3

In [47]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 2, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,110311,2.0,Ice Cream Shop,Vegetarian / Vegan Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


#### Cluster 4

In [48]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 3, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,110731,3.0,Fried Chicken Joint,Vegetarian / Vegan Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
42,111831,3.0,Fried Chicken Joint,Vegetarian / Vegan Restaurant,Drugstore,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


#### Cluster 5

In [49]:
bogota_merged.loc[bogota_merged['Cluster Labels'] == 4, bogota_merged.columns[[1] + list(range(5, bogota_merged.shape[1]))]]

,Postcode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,111511,4.0,Restaurant,BBQ Joint,Department Store,Seafood Restaurant,Shopping Mall,Pizza Place,Pie Shop,Sandwich Place,Donut Shop,Drugstore
2,111221,4.0,Soccer Field,Astrologer,Clothing Store,Bakery,Auto Garage,Creperie,Motorcycle Shop,Department Store,Cultural Center,Vegetarian / Vegan Restaurant
3,110711,4.0,Shopping Mall,Movie Theater,Vegetarian / Vegan Restaurant,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
7,111711,4.0,Café,Restaurant,Hostel,Latin American Restaurant,Coffee Shop,Italian Restaurant,Breakfast Spot,Vegetarian / Vegan Restaurant,BBQ Joint,Burger Joint
8,110211,4.0,Mountain,Vegetarian / Vegan Restaurant,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
9,110221,4.0,Gym,Hotel,Art Gallery,Athletics & Sports,Vegetarian / Vegan Restaurant,Drugstore,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall
11,111911,4.0,Ice Cream Shop,Furniture / Home Store,Movie Theater,Shopping Mall,Vegetarian / Vegan Restaurant,Department Store,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant
13,111931,4.0,Music Store,Vegetarian / Vegan Restaurant,Donut Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
18,111011,4.0,Pizza Place,Pub,Shop & Service,Gym / Fitness Center,Multiplex,Toy / Game Store,Bar,Bakery,Rental Car Location,Shopping Mall
19,111021,4.0,Gym / Fitness Center,Furniture / Home Store,Bakery,Department Store,Vegetarian / Vegan Restaurant,Drugstore,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall
